In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import datetime
import pickle
from sklearn.decomposition import PCA
#rom sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
#from geopack import geopack
import sys
import xarray as xr
import datetime as datetime
import pandas as pd

#sys.path.append('/Users/sr2/opt/anaconda3/envs/nipr_env/')
sys.path.append('/home/sachin/miniconda3/envs/torch_env')
from esn_dts_openloop import ESN, Tikhonov

In [2]:
#REPPU otuputs
#path = r'/Users/sr2/My Drive/Career/Employment/Current/JSPS/Research/Data/pwbig5min.dat'#Mac
#path = r'/home/ryuho/Documents/reddy/research/SMRAI/Data/REPPU/200/pbig5min.dat' #Ubuntu
path = r'/home/sachin/Documents/NIPR/Research/Data/REPPU/pbig5min.dat' #Server

#read the REPPU data
with open (path) as f:
    rectype = np.dtype(np.float32)
    reppu_data = np.fromfile(f, rectype) #size = 109,900,800

In [3]:
mhd_data = pd.read_csv('mhd_dates.csv')
expanded_dt = pd.concat([pd.Series(pd.date_range(start, end)) 
        for start, end in zip(mhd_data['start'], mhd_data['end'])])

In [4]:
# Reshape the array to have dimensions (days, time (5-min intervals), lat, lon)
#288 will become 1440 at 1-min resolution in SMRAI v3.0
data_reshaped = reppu_data.reshape(len(expanded_dt), 288, 30, 80) 

# Define coordinates
time = np.arange(288)
lat = np.linspace(53.1, 89.7, 30) #30 intervals between 53.1° to 89.7°
lon = np.linspace(1.6, 357.6, 80) #80 intervals between 1.6° to 357.6°

# Create 'dt' variable combining dates and five-minute intervals
dt = []
for day in expanded_dt:
    for t in time:
        dt.append(day + pd.Timedelta(minutes=t*5))
dt = np.array(dt) #convert from list to numpy array

# Create xarray Dataset
ds = xr.Dataset({'potential': (['dt', 'lat', 'lon'], data_reshaped.reshape(-1, 30, 80))},
                coords={'dt': dt, 'lat': lat, 'lon': lon})

# Add potential and units
ds['potential'].attrs['units'] = 'kV'
ds['potential'] = ds['potential'] * 1e-3 # Convert to kV
ds

<xarray.Dataset> Size: 440MB
Dimensions:    (dt: 45792, lat: 30, lon: 80)
Coordinates:
  * dt         (dt) datetime64[ns] 366kB 2021-05-10 ... 2022-07-31T23:55:00
  * lat        (lat) float64 240B 53.1 54.36 55.62 56.89 ... 87.18 88.44 89.7
  * lon        (lon) float64 640B 1.6 6.106 10.61 15.12 ... 348.6 353.1 357.6
Data variables:
    potential  (dt, lat, lon) float32 440MB 0.6909 0.507 ... -22.67 -22.71

In [5]:
def create_PCA(ds, pca_comps, plot=False):

    ds = ds.copy()

    #reshape the data to be 2D
    ds_arr = ds['potential'].values
    ds_arr = ds_arr.reshape(ds_arr.shape[0], -1)

    #create PCA object with the number of components
    pca = PCA(n_components=pca_comps)
    pca.fit(ds_arr)
    pickle.dump(pca, open('/home/sachin/Documents/NIPR/Research/Data/ESN/models/ESN_potential_pca_ID25.pkl', 'wb'))

    reduced = pca.transform(ds_arr) #transform the data into the PCA space

    #return data to original space
    #inverse_transform = pca.inverse_transform(reduced)
    #print(inverse_transform.shape)

    if plot == True:
        # Plot the explained variance
        plt.figure(figsize=(10, 6))
        plt.plot(np.cumsum(pca.explained_variance_ratio_))
        plt.xlabel('Number of components')
        plt.ylabel('Explained variance')
        plt.title('Explained variance by PCA components')
        plt.show()

    ds_pca = xr.Dataset({'potential': (['dt', 'PC'], reduced)},
                        coords={'dt': ds['dt'], 'PC': np.arange(1, pca_comps+1)})

    return ds_pca, pca

pca_comps = 10
ds, pca = create_PCA(ds, pca_comps)

In [6]:
#open the omni data and merge it with the REPPU data
#omni_mhd_path = r'/Users/sr2/My Drive/Career/Employment/Current/JSPS/Research/Analysis/Apr-24/data/omni/'
omni_mhd_path = r'/home/ryuho/Documents/reddy/research/SMRAI/Data/OMNI/'

#omni_df = pd.read_csv('omni_mhd_5min.csv')
omni_df = pd.read_csv('omni_add-feats_mhd_5min.csv')
#omni_df = pd.read_csv(omni_mhd_path+'omni_mhd_5min.csv')
omni_df.set_index('dt', inplace=True) #set the datetime as the index
omni_df = omni_df.dropna()

omni_ds = xr.Dataset(omni_df)
omni_ds['dt'] = pd.to_datetime(omni_ds['dt']) #convert the index to datetime

#merge OMNI with REPPU data
reppu_omni_ds = ds.merge(omni_ds, join='inner')
reppu_omni_ds.sortby('dt')

reppu_omni_ds = reppu_omni_ds.drop_vars(['T', 'E', 'doy','BSN_x','SYM_H','F10.7','Kp','Mach_num','Pressure'])
#reppu_omni_ds = reppu_omni_ds.drop_vars(['flow_speed','proton_density'])
reppu_omni_ds

<xarray.Dataset> Size: 4MB
Dimensions:         (dt: 42158, PC: 10)
Coordinates:
  * dt              (dt) datetime64[ns] 337kB 2021-05-10 ... 2022-07-31T23:55:00
  * PC              (PC) int64 80B 1 2 3 4 5 6 7 8 9 10
Data variables:
    potential       (dt, PC) float32 2MB 407.8 -135.0 -29.71 ... -17.34 17.39
    BY_GSE          (dt) float64 337kB 4.72 4.35 4.61 4.65 ... -8.77 -9.31 -8.22
    BZ_GSE          (dt) float64 337kB 6.24 6.55 6.56 6.89 ... -0.74 -4.03 -4.71
    flow_speed      (dt) float64 337kB 344.0 344.0 345.0 ... 454.0 465.0 478.0
    proton_density  (dt) float64 337kB 12.11 11.44 12.65 ... 8.03 7.08 6.84
    tilt_angle      (dt) float64 337kB 0.253 0.2497 0.2465 ... 0.2744 0.271

In [7]:
dates_to_remove = pd.date_range(start='2021-05-10 00:00:00', end='2021-05-10 23:55:00', freq='5min')
date_1_ds = reppu_omni_ds.where(reppu_omni_ds['dt'].isin(dates_to_remove), drop=True)
mask = ~reppu_omni_ds['dt'].isin(dates_to_remove)
date1 = reppu_omni_ds.where(mask, drop=True)

dates_to_remove2 = pd.date_range(start='2021-10-11 00:00:00', end='2021-10-11 23:55:00', freq='5min')
date_2_ds = date1.where(date1['dt'].isin(dates_to_remove2), drop=True)
mask2 = ~date1['dt'].isin(dates_to_remove2)
date2 = date1.where(mask2, drop=True)

dates_to_remove3 = pd.date_range(start='2022-01-30 00:00:00', end='2022-01-30 23:55:00', freq='5min')
date_3_ds = date2.where(date2['dt'].isin(dates_to_remove3), drop=True)
mask3 = ~date2['dt'].isin(dates_to_remove3)
reppu_omni_ds = date2.where(mask3, drop=True)
reppu_omni_ds

<xarray.Dataset> Size: 4MB
Dimensions:         (dt: 41303, PC: 10)
Coordinates:
  * dt              (dt) datetime64[ns] 330kB 2021-05-11 ... 2022-07-31T23:55:00
  * PC              (PC) int64 80B 1 2 3 4 5 6 7 8 9 10
Data variables:
    potential       (dt, PC) float32 2MB 288.5 -78.05 -63.97 ... -17.34 17.39
    BY_GSE          (dt) float64 330kB 4.2 4.22 4.19 3.78 ... -8.77 -9.31 -8.22
    BZ_GSE          (dt) float64 330kB -0.59 -0.7 -0.92 ... -0.74 -4.03 -4.71
    flow_speed      (dt) float64 330kB 340.0 343.0 344.0 ... 454.0 465.0 478.0
    proton_density  (dt) float64 330kB 8.97 8.45 8.55 8.73 ... 8.03 7.08 6.84
    tilt_angle      (dt) float64 330kB 0.2575 0.2542 0.2509 ... 0.2744 0.271

Slices for 1-min

In [8]:
# Define the slice ranges for train and test data
#train_slice = slice(0, 182880)  #182880 / 24 hr / 60 min = 127 days = 80% of the data
#test_slice = slice(182880, None)  # 46080 / 24 hr / 60 min = 32 days = 20% of the data

# Create ds_train and ds_test datasets
#ds_train = reppu_omni_ds.isel(dt=train_slice)
#ds_test = reppu_omni_ds.isel(dt=test_slice)

Slices for 5-mins

In [9]:
# Define the slice ranges for train and test data
train_slice = slice(0, 36576)  #36576 / 24 / 12 = 127 days = 80% of the data
test_slice = slice(36576, None)  # 9216 / 24 / 12 = 32 days = 20% of the data

#train_slice = slice(0, 38880)  #38880 / 24 / 12 = 135 days = 85% of the data
#test_slice = slice(38880, None)  # 6192 / 24 / 12 = 24 days = 15% of the data

# Create ds_train and ds_test datasets
ds_train = reppu_omni_ds.isel(dt=train_slice)
ds_test = reppu_omni_ds.isel(dt=test_slice)

In [10]:
#training data
X_train = ds_train.drop_vars('potential').to_array().values.T
y_train = ds_train['potential'].values

#test data
X_test = ds_test.drop_vars('potential').to_array().values.T
y_test = ds_test['potential'].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((36576, 5), (36576, 10), (4727, 5), (4727, 10))

Initial Parameters

In [11]:
'''
input_dim = X_train.shape[1] #number of input features
output_dim = y_train.shape[1] #number of output features
reservoir_size = 250 #number of reservoir neurons
density = 0.1 #connection density
input_scaling = 0.1 #scaling of input weights
spectral_radius = 0.99 #spectral radius of the recurrent matrix
beta = 1e-3 #regularization factor
'''

'\ninput_dim = X_train.shape[1] #number of input features\noutput_dim = y_train.shape[1] #number of output features\nreservoir_size = 250 #number of reservoir neurons\ndensity = 0.1 #connection density\ninput_scaling = 0.1 #scaling of input weights\nspectral_radius = 0.99 #spectral radius of the recurrent matrix\nbeta = 1e-3 #regularization factor\n'

Optimised Hyperparameters

In [12]:
input_dim = X_train.shape[1] #number of input features
output_dim = y_train.shape[1] #number of output features
reservoir_size = 450 #number of reservoir neurons
density = 0.2 #connection density
input_scaling = 0.1 #scaling of input weights
spectral_radius = 0.99 #spectral radius of the recurrent matrix
beta = 1e-2 #regularization factor

In [13]:
model = ESN(input_dim, output_dim, reservoir_size, density, input_scaling, spectral_radius)
optimizer = Tikhonov(input_dim, reservoir_size, output_dim, beta=beta)
model.train(X_train, y_train, optimizer, trans_len=32)

array([[ 175.8067162 ,  305.04261119, -398.32383653, ..., -599.39910493,
        -577.2672885 , -180.408164  ],
       [ 180.14775422,  308.91590552, -406.24757252, ..., -606.59308946,
        -584.21282127, -182.28334811],
       [ 180.59710962,  310.14554503, -407.79117774, ..., -608.60041877,
        -585.96167531, -183.17417481],
       ...,
       [ 191.77928331,  340.87312117, -458.57350182, ..., -676.65109551,
        -655.80922515, -221.24683456],
       [ 188.09794988,  333.17182584, -448.75164135, ..., -663.84013395,
        -642.14035514, -216.53744207],
       [ 188.69594836,  334.07591985, -449.88156679, ..., -665.37116707,
        -643.95120175, -217.00509346]])

In [14]:
#save or load model
with open('/home/sachin/Documents/NIPR/Research/Data/ESN/models/SMRAI2_potential_ID25.pkl', 'wb') as f:
    pickle.dump(model, f)

#load model
#with open('SMRAI2_current_SR.pkl', 'rb') as f:
#    model = pickle.load(f)

In [15]:
#make prediction on the test data
y_pred = model.predict(X_test)

In [16]:
#make prediction on the test data
y_pred = model.predict(X_test)

#inverse transform the data / return to the original space
y_pred_orig = pca.inverse_transform(y_pred)
y_pred_orig = y_pred_orig.reshape(-1, 30, 80) #reshape the prediction to lat, lon
y_test_orig = pca.inverse_transform(y_test)
y_test_orig = y_test_orig.reshape(-1, 30, 80) #reshape the prediction to lat, lon

In [17]:
ds_test['potential_pred'] = (('dt', 'lat', 'lon'), y_pred_orig)
ds_test['potential_pred'].attrs['units'] = 'kV'
ds_test['potential_test'] = (('dt', 'lat', 'lon'), y_test_orig)
ds_test['potential_test'].attrs['units'] = 'kV'

#remove data variable 'potential'
#ds_test = ds_test.drop_vars('potential')
#ds_test = ds_test.drop_dims('PC')
#ds_test = ds_test.set_coords(['lat', 'lon'])

ds_test['err_abs'] = np.abs(ds_test['potential_pred'] - ds_test['potential_test'])
#ds_test['err'] = ds_test['potential_pred'] - ds_test['potential']
#ds_test['err_pct'] = ds_test['err'] / ds_test['potential'] * 100
#ds_test['MLT'] = np.mod(ds_test['lon'] / 15, 24)

ds_test

<xarray.Dataset> Size: 227MB
Dimensions:         (dt: 4727, PC: 10, lat: 30, lon: 80)
Coordinates:
  * dt              (dt) datetime64[ns] 38kB 2022-07-14T21:10:00 ... 2022-07-...
  * PC              (PC) int64 80B 1 2 3 4 5 6 7 8 9 10
Dimensions without coordinates: lat, lon
Data variables:
    potential       (dt, PC) float32 189kB 729.4 310.3 -151.0 ... -17.34 17.39
    BY_GSE          (dt) float64 38kB 6.9 6.66 6.71 6.45 ... -8.77 -9.31 -8.22
    BZ_GSE          (dt) float64 38kB -2.58 -2.45 -2.25 ... -0.74 -4.03 -4.71
    flow_speed      (dt) float64 38kB 383.0 360.0 353.0 ... 454.0 465.0 478.0
    proton_density  (dt) float64 38kB 0.48 0.47 0.4 0.38 ... 7.12 8.03 7.08 6.84
    tilt_angle      (dt) float64 38kB 0.4447 0.4414 0.4381 ... 0.2744 0.271
    potential_pred  (dt, lat, lon) float64 91MB 2.406 2.079 ... -22.23 -22.26
    potential_test  (dt, lat, lon) float32 45MB 7.17 6.552 ... -22.73 -22.77
    err_abs         (dt, lat, lon) float64 91MB 4.765 4.473 ... 0.4973 0.5054

In [18]:
#save ds_test
#ds_test.to_netcdf('SMRAI2_potential_ID15.nc')

In [19]:
def nan_count(ds, var):
    #count number of NaNs in a data variable
    con_data = ds[var].values
    nan_count = np.count_nonzero(np.isnan(con_data))
    nan_ratio = nan_count / con_data.size
    not_nan = con_data.size - nan_count
    return not_nan, np.round(nan_ratio, 2)

#nan_count(ds_test, 'Pressure')

In [20]:
potential = ds_test['potential_test'].values
potential = potential.reshape(-1, 30, 80)

pred_pot = ds_test['potential_pred'].values
pred_pot = pred_pot.reshape(-1, 30, 80)


def skill(m, o):
    skill = 1 - (np.sum((m - o)**2) / 
            np.sum((o - np.mean(o))**2))

    return skill             

#RMSE = np.sqrt(np.mean((potential - pred_pot)**2))
#NRMSE = RMSE / (np.max(potential) - np.min(potential))
MAE = np.mean(np.abs(potential - pred_pot))
NMAE = MAE / np.std(potential)
PE = skill(pred_pot, potential)
R = np.corrcoef(potential.flatten(), pred_pot.flatten())[0, 1]

#print(f'RMSE: {RMSE:.1f} kV')
#print(f'NRMSE: {NRMSE:.2f}')
print(f'MAE: {MAE:.2f} kV')
print(f'NMAE: {NMAE:.3f}')
print(f'R: {R:.2f}/1')
print(f'PE: {PE:.2f}/1')

MAE: 4.10 kV
NMAE: 0.344
R: 0.87/1
PE: 0.75/1


In [21]:
#plot hist of AE
sns.histplot(ds_test['err'].values.flatten(), bins=100)
plt.xlabel('Absolute Error (kV)')
plt.ylabel('Frequency')
plt.yscale('log')

KeyError: "No variable named 'err'. Variables on the dataset include ['potential', 'BY_GSE', 'BZ_GSE', 'flow_speed', 'proton_density', ..., 'dt', 'PC', 'potential_pred', 'potential_test', 'err_abs']"

In [ ]:
# Define common parameters
theta = np.linspace(0, 360, 80) - 90
theta_rad = theta / 360 * 2 * np.pi
r = 90 - np.linspace(53.1, 89.7, 30)
shrink = .5

# Plot true values
abs_error = (ds_test['err_abs'].mean('dt').values)

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(6,5))

c_true = ax.contourf(theta_rad, r, abs_error, levels=np.linspace(0, 5, 40), cmap='rainbow', extend='both')
ax.set_ylim([0, 37])
ax.set_yticks([0, 10, 20, 30])
ax.set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
ax.set_xlim([-np.pi, np.pi])
ax.set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
ax.set_xticklabels(["21", "0 MLT \nMidnight", "3", "6 \n  Dawn", "9", "12 \nMidday", "15", "18 \nDusk"])
ax.set_title(f'SMRAI 2.1 (ESN)\nAbsolute Error, ID25')
plt.colorbar(c_true, ax=ax, label='Potential (Error) [kV]', shrink=shrink, pad=0.15, ticks=np.arange(0,6,1), orientation='horizontal')
ax.grid(True, linestyle='-', alpha=0.6)
#plt.tight_layout()
plt.savefig('smrai_esn_ID25.png', dpi=400, bbox_inches='tight')

In [ ]:
#plot potential on 2022-07-11, integrtaed over lat and lon

#2022-07-12 is worst


#2022-07-31 is okay
#2022-07-25 is okay

#2022-07-29 is best
#2022-07-30 is best
date = '2022-07-31'

#date_range = pd.date_range('2022-07-29', '2022-07-30', freq = 'm')

#extract SYM-H values
symh = ds_pred['Kp'].sel(dt=date)

symh_hourly = pd.DataFrame(symh.values, columns=['Kp'])
symh_hourly['dt'] = symh['dt'].values
symh_hourly.set_index('dt', inplace=True)
symh_hourly = symh_hourly.resample('3h').mean()

fig, ax = plt.subplots(figsize=(6, 4))
ds_pred['potential'].sel(dt=date).mean(['lat', 'lon']).plot(label='SMRAI2 (Prediction)', color='#648FFF')
ds_test['potential'].sel(dt=date).mean(['lat', 'lon']).plot(label='REPPU (G. Truth)', color='#FFB000')

rmse = np.sqrt(np.mean((ds_pred['potential'].sel(dt=date).values - ds_test['potential'].sel(dt=date).values)**2))
r = np.corrcoef(ds_pred['potential'].sel(dt=date).values.flatten(), ds_test['potential'].sel(dt=date).values.flatten())[0, 1]

def skill(m, o):
    skill = 1 - (np.sum((m - o)**2) / 
            np.sum((o - np.mean(o))**2))

    return skill  

skill_val = skill(ds_pred['potential'].sel(dt=date).values, ds_test['potential'].sel(dt=date).values)


'''
i = plt.annotate(f'RMSE = {rmse:.2f} kV', xy=(0.75, 0.05), xycoords='axes fraction')
j = plt.annotate(f'R = {r:.2f}', xy=(0.75, 0.12), xycoords='axes fraction')
k = plt.annotate(f'Skill = {skill_val:.2f}', xy=(0.75, 0.19), xycoords='axes fraction')

t.set_bbox(dict(facecolor='white', alpha=1, edgecolor='white'))
i.set_bbox(dict(facecolor='white', alpha=1, edgecolor='white'))
j.set_bbox(dict(facecolor='white', alpha=1, edgecolor='white'))'''

annotations = [
    (f'RMSE = {rmse:.1f} kV', (0.75, 0.05)),
    (f'R = {r:.2f}', (0.75, 0.15)),
    (f'Skill = {skill_val:.2f}', (0.75, 0.25))
]

for annotation, position in annotations:
    annot = plt.annotate(annotation, xy=position, xycoords='axes fraction')
    annot.set_bbox(dict(facecolor='white', alpha=0.7, edgecolor='white'))

#add SYM-H label
#plt.annotate('Kp:', xy=(-0.04, -0.25), xycoords='axes fraction'
#            , ha='center', va='center', color='k')
#add sym_hourly values beneath the x ticks
for i, val in enumerate(symh_hourly['Kp'].values):
    plt.annotate(f'{val:.1f}', xy=((0.39+i)/8.8, -0.25), xycoords='axes fraction', ha='center', va='center')

#remove top and right spines


plt.legend()
plt.title(f'Polar Cap Potential \n{date}',pad=8)
plt.xlabel(' ')
phi = r'$\mathrm{\phi}$'
plt.ylabel(f'Potential ({phi}) [kV]')
plt.grid(True)
plt.tight_layout()
#plt.savefig('/Users/sr2/My Drive/Career/Employment/Current/JSPS/Research/Analysis/Apr-24/plots/SMRAI3/potential/2022-07-31/rmse.png', dpi=300)

In [ ]:
#loop through all dates in the test and pred dataset and calculate the RMSE then put into a df
rmse_list = []
#r_list = []
#skill_list = []
for date in ds_test['dt']:
    rmse = np.sqrt(np.mean((ds_pred['potential'].sel(dt=date).values - ds_test['potential'].sel(dt=date).values)**2))
    #r = np.corrcoef(ds_pred['potential'].sel(dt=date).values.flatten(), ds_test['potential'].sel(dt=date).values.flatten())[0, 1]
    #skill_val = skill(ds_pred['potential'].sel(dt=date).values, ds_test['potential'].sel(dt=date).values)
    rmse_list.append(rmse)
    #r_list.append(r)
    #skill_list.append(skill_val)

rmse_df = pd.DataFrame(rmse_list, columns=['RMSE'])
rmse_df['dt'] = ds_test['dt']

rmse_df['date'] = rmse_df['dt'].dt.date
rmse_df['hr'] = rmse_df['dt'].dt.hour
#rmse_df['Kp'] = ds_test['Kp'].values
rmse_df

In [ ]:
rmse_df.groupby('date').mean().sort_values(by='RMSE', ascending=True)

In [ ]:
# Define the date
date = "2022-07-29"
ind = 144

# Extract true and predicted values
pot_true = ds_test['potential_test'].sel(dt=date).values[ind]
pot_pred = ds_test['potential_pred'].sel(dt=date).values[ind]

# Calculate the difference
abs_error  = pot_true - pot_pred    
#pct_error = ds_test['Error_pct'].sel(dt=date).values[ind]

# Set up the plot
fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(14, 4))

# Define common parameters
theta = np.linspace(0, 360, 80) - 90
theta_rad = theta / 360 * 2 * np.pi
r = 90 - np.linspace(53.1, 89.7, 30)
shrink = .3
10]', shrink=shrink, pad=0.1, ticks=np.arange(-30, 31, 10), orientation='horizontal')

# Plot predicted values
c_pred = axs[1].contourf(theta_rad, r, -pot_pred, levels=np.linspace(-30, 30, 50), cmap='PuOr', extend='both')
axs[1].set_ylim([0, 30])
axs[1].set_yticks([0, 10, 20, 30])
axs[1].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[1].set_xlim([-np.pi, np.pi])
axs[1].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[1].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
axs[1].set_title(f'Predicted Potential \n (SMRAI)')
plt.colorbar(c_pred, ax=axs[1], label='Potential [kV]', shrink=shrink, pad=0.1, ticks=np.arange(-30, 31, 10), orientation='horizontal')

# Plot difference
c_diff = axs[2].contourf(theta_rad, r, abs_error, levels=np.linspace(0, 10, 50), cmap='viridis', extend='both')
axs[2].set_ylim([0, 30])
axs[2].set_yticks([0, 10, 20, 30])
axs[2].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[2].set_xlim([-np.pi, np.pi])
axs[2].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[2].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
axs[2].set_title('Difference \n(True - Predicted)')
plt.colorbar(c_diff, ax=axs[2], label='Absolute Error [kV]', shrink=shrink, pad=0.1, ticks=np.arange(0, 11,2), orientation='horizontal')

plt.subplots_adjust(wspace=-0.7)  # Adjust horizontal space between subplots

plt.tight_layout()
plt.show()

In [ ]:
# Define the date
date = "2022-07-31"

# Extract true and predicted values
pot_true = ds_test['potential_test'].sel(dt=date).values[0]
pot_pred = ds_test['potential_pred'].sel(dt=date).values[0]

# Calculate the difference
diff = pot_true - pot_pred

# Set up the plot
fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(14, 4))

# Define common parameters
theta = np.linspace(0, 360, 80) - 90
theta_rad = theta / 360 * 2 * np.pi
r = 90 - np.linspace(53.1, 89.7, 30)
shrink = .3

# Plot true values
c_true = axs[0].contourf(theta_rad, r, -pot_true, levels=np.linspace(-30, 30, 50), cmap='PiYG', extend='both')
axs[0].set_ylim([0, 30])
axs[0].set_yticks([0, 10, 20, 30])
axs[0].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[0].set_xlim([-np.pi, np.pi])
axs[0].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[0].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
axs[0].set_title(f'True Potential \n (REPPU)')
plt.colorbar(c_true, ax=axs[0], label='Potential [kV]', shrink=shrink, pad=0.1, ticks=np.arange(-30, 31, 10), orientation='horizontal')

# Plot predicted values
c_pred = axs[1].contourf(theta_rad, r, -pot_pred, levels=np.linspace(-30, 30, 50), cmap='PiYG', extend='both')
axs[1].set_ylim([0, 30])
axs[1].set_yticks([0, 10, 20, 30])
axs[1].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[1].set_xlim([-np.pi, np.pi])
axs[1].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[1].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
axs[1].set_title(f'Predicted Potential \n (SMRAI)')
plt.colorbar(c_pred, ax=axs[1], label='Potential [kV]', shrink=shrink, pad=0.1, ticks=np.arange(-30, 31, 10), orientation='horizontal')

# Plot difference
c_diff = axs[2].contourf(theta_rad, r, -diff, levels=np.linspace(-30, 30, 50), cmap='bwr', extend='both')
axs[2].set_ylim([0, 30])
axs[2].set_yticks([0, 10, 20, 30])
axs[2].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[2].set_xlim([-np.pi, np.pi])
axs[2].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[2].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
axs[2].set_title('Difference \n(True - Predicted)')
plt.colorbar(c_diff, ax=axs[2], label='Difference [kV]', shrink=shrink, pad=0.1, ticks=np.arange(-30, 31, 10), orientation='horizontal')

plt.subplots_adjust(wspace=-0.7)  # Adjust horizontal space between subplots

plt.tight_layout()
plt.show()


In [ ]:

# Select the date
date = "2022-07-31"

# Extract times from dt dimension info
times = ds_test.sel(dt=date)['dt'].values

# Define common parameters
theta = np.linspace(0, 360, 80) - 90
theta_rad = theta / 360 * 2 * np.pi
r = 90 - np.linspace(53.1, 89.7, 30)
shrink = .28
pad = 0.18
t_pad = 10
phi = r"$\mathrm{\phi}$"

# Loop through each time step
for i, time in enumerate(times):
    # Extract true and predicted values for the current time
    pot_true = ds_test['potential'].sel(dt=date).values[i]
    pot_pred = ds_pred['potential'].sel(dt=date).values[i]

    # Calculate the difference
    diff = pot_true - pot_pred

    # Set up the plot
    fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(14, 4))

    # Plot true values
    c_true = axs[0].contourf(theta_rad, r, -pot_true, levels=np.linspace(-30, 30, 50), cmap='PuOr', extend='both')
    axs[0].set_ylim([0, 30])
    axs[0].set_yticks([0, 10, 20, 30])
    axs[0].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
    axs[0].set_xlim([-np.pi, np.pi])
    axs[0].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
    axs[0].set_xticklabels(["21", "0 MLT \nMidnight", "3", "6", "9", "Midday \n12", "15", "18"])
    axs[0].set_title(f'True Potential \n (REPPU)',pad=t_pad)
    axs[0].grid(True, alpha=0.7)
    plt.colorbar(c_true, ax=axs[0], label=f'Potential ({phi}) [kV]', shrink=shrink, pad=pad, ticks=np.arange(-30, 31, 10), orientation='horizontal')

    # Plot predicted values
    c_pred = axs[1].contourf(theta_rad, r, -pot_pred, levels=np.linspace(-30, 30, 50), cmap='PuOr', extend='both')
    axs[1].set_ylim([0, 30])
    axs[1].set_yticks([0, 10, 20, 30])
    axs[1].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
    axs[1].set_xlim([-np.pi, np.pi])
    axs[1].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
    axs[1].set_xticklabels(["21", "0 MLT \nMidnight", "3", "6", "9", "Midday \n12", "15", "18"])
    axs[1].set_title('Predicted Potential \n (SMRAI)',pad=t_pad)
    axs[1].grid(True, alpha=0.7)
    plt.colorbar(c_pred, ax=axs[1], label=f'Potential ({phi}) [kV]', shrink=shrink, pad=pad, ticks=np.arange(-30, 31, 10), orientation='horizontal')

    # Plot difference
    c_diff = axs[2].contourf(theta_rad, r, -diff, levels=np.linspace(-30, 30, 50), cmap='bwr', extend='both')
    axs[2].set_ylim([0, 30])
    axs[2].set_yticks([0, 10, 20, 30])
    axs[2].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
    axs[2].set_xlim([-np.pi, np.pi])
    axs[2].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
    axs[2].set_xticklabels(["21", "0 MLT \nMidnight", "3", "6", "9", "Midday \n12", "15", "18"])
    axs[2].set_title(f'Difference \n(True - Predicted)',pad=t_pad)
    axs[2].grid(True, alpha=0.7)
    plt.colorbar(c_diff, ax=axs[2], label=f'Difference ({phi}) [kV]', shrink=shrink, pad=pad, ticks=np.arange(-30, 31, 10), orientation='horizontal')

    plt.subplots_adjust(wspace=-0.8)  # Adjust horizontal space between subplots

    plt.tight_layout()

    dt = pd.to_datetime(time)
    str_time = dt.strftime('%Y-%m-%d %H-%M')

    plt.annotate(f'{str_time}', xy=(-1.45, -0.7), xycoords='axes fraction', fontsize=12)

    # Save the figure with the timestamp in the title
    #plt.savefig(f'/Users/sr2/My Drive/Career/Employment/Current/JSPS/Research/Analysis/Apr-24/plots/SMRAI3/potential/2022-07-31/potential_plot_{str_time}.png', dpi=300, bbox_inches='tight')
    
    # Close the figure to free up memory
    #plt.close(fig)

    break